In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# View all of the classes that automap found
# View all of the classes that automap found
classes = Base.classes
print(classes.keys())


In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.
recent_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()[0]


In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.
one_year_ago = dt.datetime.strptime(recent_date, '%Y-%m-%d') - dt.timedelta(days=365)

# Perform a query to retrieve the data and precipitation scores
precipitation_data = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= one_year_ago).all()


# Save the query results as a Pandas DataFrame and set the index to the date column
precipitation_df = pd.DataFrame(precipitation_data, columns=['date', 'precipitation'])
precipitation_df.set_index('date', inplace=True)


# Sort the dataframe by date
precipitation_df = precipitation_df.sort_values('date')

# Use Pandas Plotting with Matplotlib to plot the data
ax = precipitation_df.plot(kind='bar', width=3, figsize=(12,8))
ax.xaxis.set_major_formatter(plt.NullFormatter())
plt.xticks([])
plt.xlabel('Date')
plt.ylabel('Inches')
plt.legend(['Precipitation'], loc='upper center')
plt.title("Precipitation in Hawaii (Last 12 Months)")
plt.show()



In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
print(precipitation_df.describe())

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset
station_count = session.query(func.count(Station.station)).scalar()
print(f'There are {station_count} stations in this dataset.\n')


In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.
most_active_stations = session.query(Measurement.station, func.count(Measurement.station)).\
    group_by(Measurement.station).order_by(func.count(Measurement.station).desc()).all()

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
temps = session.query(func.min(Measurement.tobs), func.max(Measurement.tobs), func.avg(Measurement.tobs)).\
        filter(Measurement.Station == station).all()

# Print the results
print(f"Temperature for Station {station}")
print(f"Lowest Temperature: {temps[0][0]}")
print(f"Highest Temperature: {temps[0][1]}")
print(f"Average Temperature: {temps[0][2]}")


In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
results = session.query(Measurement.tobs).\
            filter(Measurement.Station == most_active_station).\
            filter(Measurement.date >= one_year_ago).all()

# Convert the query results to a list
temperatures = list(np.ravel(results))

# Create a histogram using the temperature data and set the number of bins to 12
plt.hist(temperatures, bins=12, label='tobs')

# Add labels and a legend to the plot
plt.title(f'Temperature Observations for Station {most_active_station} ({one_year_ago} to {recent_date})')
plt.xlabel('Temperature (F)')
plt.ylabel('Frequency')
plt.legend()

# Show the plot
plt.show()


# Close session

In [ ]:
# Close Session
session.close()